Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
data = pd.read_csv('salary-train.csv')
data_test = pd.read_csv('salary-test-mini.csv')

data['FullDescription'] = (data['FullDescription'].apply(lambda text: text.lower())).replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = (data_test['FullDescription'].apply(lambda text: text.lower())).replace('[^a-zA-Z0-9]', ' ', regex = True)

In [15]:
y_train = float(data['SalaryNormalized'].values)

TypeError: only length-1 arrays can be converted to Python scalars

Проведите предобработку:

Приведите тексты к нижнему регистру (text.lower()).

Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text).

Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
tf_vec = TfidfVectorizer(min_df=5)
X_train = tf_vec.fit_transform(data['FullDescription'])
X_test = tf_vec.transform(data_test['FullDescription'])

In [7]:
data['LocationNormalized'] = data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'] = data['ContractTime'].fillna('nan', inplace=True)
#тестовая
data_test['LocationNormalized'] = data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'] = data_test['ContractTime'].fillna('nan', inplace=True)

In [8]:
from sklearn.feature_extraction import DictVectorizer

In [9]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [11]:
from scipy.sparse import hstack

In [12]:
X = hstack([X_train, X_train_categ])

In [13]:
from sklearn.linear_model import Ridge

In [14]:
clf = Ridge(alpha=1.0, random_state=241)
clf.fit(X, y_train)
Ts=clf.predict(X_test_categ)
print ts

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').